In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from math import *

In [2]:
def get_expected_pseudoranges(x_est, X, Y, Z, B):
    p = np.zeros_like(X)
    for i in range(len(X)):
        p[i] = np.sqrt((X[i]-x_est[0])**2 +
            (Y[i]-x_est[1])**2 + (Z[i]-x_est[2])**2) - B[i]
    return p

In [3]:
def get_geometry_matrix(x_est, X, Y, Z):
    G = np.ones((len(X),4))
    for i in range(len(X)):
        d = np.sqrt((X[i]-x_est[0])**2 +
            (Y[i]-x_est[1])**2 + (Z[i]-x_est[2])**2)
        G[i,0] = -(X[i]-x_est[0])/d
        G[i,1] = -(Y[i]-x_est[1])/d
        G[i,2] = -(Z[i]-x_est[2])/d
    return G

In [4]:
def solve_pos(x0, X, Y, Z, B, prange):
    dxm = 100
    x = x0
    n = 0
    while dxm>1:
        G = get_geometry_matrix(x, X, Y, Z)
        H = np.linalg.inv(np.matmul(G.T,G))
        xdop = np.sqrt(H[0,0])
        ydop = np.sqrt(H[1,1])
        zdop = np.sqrt(H[2,2])
        p_0 = get_expected_pseudoranges(x, X, Y, Z, B)
        dp = prange - p_0
        dx = np.matmul(np.matmul(np.linalg.inv(np.matmul(G.T,G)),G.T),dp)
        dxm = np.linalg.norm(dx[:3])
        x = x+dx[:3]
        n += 1
    return x, n,xdop,ydop,zdop

In [5]:
def E_next(E,e,M):
    return (e*(np.sin(E)-E*np.cos(E))+M)/(1-e*np.cos(E))

In [6]:
def get_sat_ECEF(ephem, tx_time):
    a = ephem['sqrtA']**2
    mu = 3.986005E14
    Omega_e = 7.2921151467E-5
    n = np.sqrt(mu/a**3) + ephem['DeltaN']
    t = tx_time
    tk = t-float(ephem['Toe'])
    if tk<-302400:
        tk = tk+604800
    elif tk>302400:
        tk = tk-604800
    Mk = ephem['M0'] + n*tk
    e = ephem['Eccentricity']
    Ek = Mk
    for i in range(20):
        Ek = E_next(Ek,e,Mk)
    sinvk = np.sqrt(1-e**2)*np.sin(Ek)/(1-e*np.cos(Ek))
    cosvk = (np.cos(Ek)-e)/(1-e*np.cos(Ek))
    vk = atan2(sinvk,cosvk)
    phik = vk + ephem['omega']
    dphik = ephem['Cus']*np.sin(2*phik) + ephem['Cuc']*np.cos(2*phik)
    uk = phik+dphik
    drk = ephem['Crs']*np.sin(2*phik) + ephem['Crc']*np.cos(2*phik)
    dik = ephem['Cis']*np.sin(2*phik) + ephem['Cic']*np.cos(2*phik)
    Omegak = ephem['Omega0']-Omega_e*t+ephem['OmegaDot']*tk
    rk = a*(1-e*np.cos(Ek))+drk
    ik = ephem['Io']+ephem['IDOT']*tk+dik
    xp = rk*np.cos(uk)
    yp = rk*np.sin(uk)
    x_ECEF = xp*np.cos(Omegak)-yp*np.cos(ik)*np.sin(Omegak)
    y_ECEF = xp*np.sin(Omegak)+yp*np.cos(ik)*np.cos(Omegak)
    z_ECEF = yp*np.sin(ik)
    return x_ECEF,y_ECEF,z_ECEF,ephem['Svid']

In [7]:
def lla_to_ecef_1(lat, lon, alt):
    # see http://www.mathworks.de/help/toolbox/aeroblks/llatoecefposition.html
    rad = np.float64(6378137.0)        # Radius of the Earth (in meters)
    f = np.float64(1.0/298.257223563)  # Flattening factor WGS84 Model
    cosLat = np.cos(lat*np.pi/180)
    sinLat = np.sin(lat*np.pi/180)
    FF     = (1.0-f)**2
    C      = 1/np.sqrt(cosLat**2 + FF * sinLat**2)
    S      = C * FF

    x = (rad * C + alt)*cosLat * np.cos(lon*np.pi/180)
    y = (rad * C + alt)*cosLat * np.sin(lon*np.pi/180)
    z = (rad * S + alt)*sinLat
    return x, y, z

### Relative positioning

In [8]:
f = open('gnss_log_2022_03_07_19_43_57.txt')
lat = []
long = []
alt = []
for i in f.readlines():
    a = i.split(',')
    if a[0] == 'Fix':
        lat.append(float(a[2]))
        long. append(float(a[3]))
        alt.append(float(a[4]))
lat = np.array(lat)
long = np.array(long)
alt = np.array(alt)

In [9]:
f = open('gnss_log_2022_03_07_19_43_57.txt')
rtime = []
for i in f.readlines():
    a = i.split(',')
    if a[0] == 'Raw':
        rtime.append(float(a[14]))
rtime = np.round(np.array(rtime)*10**-9)

In [10]:
xe = np.zeros(len(lat))
ye = np.zeros(len(lat))
ze = np.zeros(len(lat))
for i in range(len(lat)):
    xe[i],ye[i],ze[i] = lla_to_ecef_1(lat[i],long[i],alt[i])

In [11]:
f = open('gnss_log_2022_03_07_19_45_15.txt')
lat1 = []
long1 = []
alt1 = []
for i in f.readlines():
    a = i.split(',')
    if a[0] == 'Fix':
        lat1.append(float(a[2]))
        long1. append(float(a[3]))
        alt1.append(float(a[4]))
lat1 = np.array(lat1)
long1 = np.array(long1)
alt1 = np.array(alt1)

In [12]:
f = open('gnss_log_2022_03_07_19_45_15.txt')
rtime1 = []
for i in f.readlines():
    a = i.split(',')
    if a[0] == 'Raw':
        rtime1.append(float(a[14]))
rtime1 = np.round(np.array(rtime1)*10**-9)

In [13]:
xe1 = np.zeros(len(lat1))
ye1 = np.zeros(len(lat1))
ze1 = np.zeros(len(lat1))
for i in range(len(lat1)):
    xe1[i],ye1[i],ze1[i] = lla_to_ecef_1(lat1[i],long1[i],alt1[i])

In [14]:
f = open('gnss_log_2022_03_07_19_44_25.txt')
lat2 = []
long2 = []
alt2 = []
for i in f.readlines():
    a = i.split(',')
    if a[0] == 'Fix':
        lat2.append(float(a[2]))
        long2. append(float(a[3]))
        alt2.append(float(a[4]))
lat2 = np.array(lat2)
long2 = np.array(long2)
alt2 = np.array(alt2)

In [15]:
xe2 = np.zeros(len(lat2))
ye2 = np.zeros(len(lat2))
ze2 = np.zeros(len(lat2))
for i in range(len(lat2)):
    xe2[i],ye2[i],ze2[i] = lla_to_ecef_1(lat2[i],long2[i],alt2[i])

In [16]:
np.linalg.norm(np.array([np.average(xe2)-np.average(xe1),np.average(ye2)-np.average(ye1),np.average(ze2)-np.average(ze1)]))

10.125052145214994

### Position estimation for different error modellings

In [17]:
data1 = pd.read_csv('data1.csv')

In [18]:
data2 = pd.read_csv('data2.csv')

In [19]:
truedata = pd.read_csv('goGPS_WLS.csv')

In [20]:
times = np.array(list(set(data1['millisSinceGpsEpoch'])))
len(times)

1739

In [21]:
times = np.sort(times)

In [22]:
time1 = data1[data1['millisSinceGpsEpoch']>=times[0]]
X_ECEF = np.zeros(len(time1))
Y_ECEF = np.zeros(len(time1))
Z_ECEF = np.zeros(len(time1))

In [23]:
x_1 = np.zeros(3)
x0 = np.array([0,0,0])
nr_pos = np.zeros((len(times),3))
XDOP = np.zeros(len(times))
YDOP = np.zeros(len(times))
ZDOP = np.zeros(len(times))
for i in range(len(times)):
    a = time1[time1['millisSinceGpsEpoch'] == times[i]]
    X1 = np.array(a['xSatPosM'])
    Y1 = np.array(a['ySatPosM'])
    Z1 = np.array(a['zSatPosM'])
    prange1 = np.array(a['rawPrM'])
    B1 = np.array(a['satClkBiasM']) - np.array(a['isrbM']) - np.array(a['ionoDelayM']) - np.array(a['tropoDelayM'])
    PRN1 = np.array(a['svid'])
    x_1, q,xdop,ydop,zdop = solve_pos(x0, X1, Y1, Z1, B1, prange1)
    nr_pos[i] = x_1
    XDOP[i] = xdop
    YDOP[i] = ydop
    ZDOP[i] = zdop

In [24]:
x_1 = np.zeros(3)
x0 = np.array([0,0,0])
nr_p = np.zeros((len(times),3))
XDOP = np.zeros(len(times))
YDOP = np.zeros(len(times))
ZDOP = np.zeros(len(times))
for i in range(len(times)):
    a = time1[time1['millisSinceGpsEpoch'] == times[i]]
    X1 = np.array(a['xSatPosM'])
    Y1 = np.array(a['ySatPosM'])
    Z1 = np.array(a['zSatPosM'])
    prange1 = np.array(a['rawPrM'])
    B1 = np.array(a['satClkBiasM'])
    PRN1 = np.array(a['svid'])
    x_1, q,xdop,ydop,zdop = solve_pos(x0, X1, Y1, Z1, B1, prange1)
    nr_p[i] = x_1
    XDOP[i] = xdop
    YDOP[i] = ydop
    ZDOP[i] = zdop

In [25]:
A = pd.DataFrame(nr_pos, columns=['X','Y','Z'])

In [26]:
A['time'] = np.round(times*10**-3)
A['xdop'] = XDOP
A['ydop'] = YDOP
A['zdop'] = ZDOP

In [27]:
A.to_csv('nr.csv')

In [28]:
xb2 = x_1.copy()

In [29]:
def error1(s1,x1,X,Y,Z,prange,B,prn):
    r1 = np.sqrt((x1[0]-X)**2+(x1[1]-Y)**2+(x1[2]-Z)**2)
    e1 = prange - r1
    e = {}
    for i in range(len(X)):
        e[prn[i]] = e1[i]
    return e

In [30]:
def correct1(s2,e,X,Y,Z,prange,B,prn):
    p2 = prange.copy()
    j = []
    for i in range(len(X)):
        try:
            p2[i] = prange[i] - 0.01*e[prn[i]]
        except:
            j.append(i)
    for i in j:
        X = np.delete(X,i)
        Y = np.delete(Y,i)
        Z = np.delete(Z,i)
        B = np.delete(B,i)
        p2 = np.delete(p2,i)
    x2, _ ,_,_,_= solve_pos(x0, X, Y, Z, B, p2)
    return x2,p2

In [31]:
def each_step1(x1,x2):
    e1 = error1(1,x1,X1,Y1,Z1,prange1,B1,PRN1)
    e2 = error1(2,x2,X2,Y2,Z2,prange2,B2,PRN2)
    for i in e1.keys():
        for j in e2.keys():
            if i==j:
                e1[i] = e2[j]
    x11,pr1 = correct1(1,e1,X1,Y1,Z1,prange1,B1,PRN1)
    x21,pr2 = x2,prange2
    return x11,x21,pr1,pr2

In [32]:
def each_step2(x1,x2):
    e1 = error1(1,x1,X1,Y1,Z1,prange1,B1,PRN1)
    e2 = error1(2,x2,X2,Y2,Z2,prange2,B2,PRN2)
    for i in e1.keys():
        for j in e2.keys():
            if i==j:
                e = e1[i]
                e1[i] = e2[j]
                e2[j] = e
    x11,pr1 = correct1(1,e1,X1,Y1,Z1,prange1,B1,PRN1)
    x21,pr2 = correct1(2,e2,X2,Y2,Z2,prange2,B2,PRN2)
    return x11,x21,pr1,pr2

In [33]:
def each_step3(x1,x2):
    e1 = error1(1,x1,X1,Y1,Z1,prange1,B1,PRN1)
    e2 = error1(2,x2,X2,Y2,Z2,prange2,B2,PRN2)
    for i in e1.keys():
        for j in e2.keys():
            if i==j:
                e = (e1[i]+e2[j])/2
                e1[i] = e
                e2[j] = e
    x11,pr1 = correct1(1,e1,X1,Y1,Z1,prange1,B1,PRN1)
    x21,pr2 = correct1(2,e2,X2,Y2,Z2,prange2,B2,PRN2)
    return x11,x21,pr1,pr2

In [34]:
def each_step4(x1,x2):
    e1 = error1(1,x1,X1,Y1,Z1,prange1,B1,PRN1)
    e2 = error1(2,x2,X2,Y2,Z2,prange2,B2,PRN2)
    for i in e1.keys():
        for j in e2.keys():
            if i==j:
                e = (e1[i]-e2[j])/2
                e1[i] = e
                e2[j] = -e
    x11,pr1 = correct1(1,e1,X1,Y1,Z1,prange1,B1,PRN1)
    x21,pr2 = correct1(2,e2,X2,Y2,Z2,prange2,B2,PRN2)
    return x11,x21,pr1,pr2

In [35]:
x_true = np.array([truedata['ECEF X[m]'],truedata['ECEF Y[m]'],truedata['ECEF Z[m]']]).T

In [48]:
b = 25

In [50]:
nr1_aft = np.zeros_like(nr_p)
for i in range(len(times[:300])):
    t1 = time1[time1['millisSinceGpsEpoch'] == times[i]]
    t2 = time1[time1['millisSinceGpsEpoch'] == times[b]]
    X1 = np.array(t1['xSatPosM'])
    Y1 = np.array(t1['ySatPosM'])
    Z1 = np.array(t1['zSatPosM'])
    prange1 = np.array(t1['rawPrM'])
    B1 = np.array(t1['satClkBiasM'])
    PRN1 = np.array(t1['svid'])
    X2 = np.array(t2['xSatPosM'])
    Y2 = np.array(t2['ySatPosM'])
    Z2 = np.array(t2['zSatPosM'])
    prange2 = np.array(t2['rawPrM'])
    B2 = np.array(t2['satClkBiasM'])
    PRN2 = np.array(t2['svid'])
    x1 = nr_p[i]
    x2 = nr_p[b]
    for j in range(10):
        x1,x2,_,_ = each_step1(x1,x2)
    nr1_aft[i] = x1
err = np.zeros(len(nr_p[:300]))
for i in range(len(nr_p[:300])):
    err[i] = np.linalg.norm(nr_p[i]-x_true[i])
err = err[np.isnan(err)!=True]
print(np.average(err))
err_1 = np.zeros(len(nr1_aft[:300]))
for i in range(len(nr1_aft[:300])):
    err_1[i] = np.linalg.norm(nr1_aft[i]-x_true[i])
err_1 = err_1[np.isnan(err_1)!=True]
print(np.average(err_1))

1047.201113611566
1121.3278679131847


In [39]:
nr1_aft = np.zeros_like(nr_p)
for i in range(len(times[:300])):
    t1 = time1[time1['millisSinceGpsEpoch'] == times[i]]
    t2 = time1[time1['millisSinceGpsEpoch'] == times[b]]
    X1 = np.array(t1['xSatPosM'])
    Y1 = np.array(t1['ySatPosM'])
    Z1 = np.array(t1['zSatPosM'])
    prange1 = np.array(t1['rawPrM'])
    B1 = np.array(t1['satClkBiasM'])
    PRN1 = np.array(t1['svid'])
    X2 = np.array(t2['xSatPosM'])
    Y2 = np.array(t2['ySatPosM'])
    Z2 = np.array(t2['zSatPosM'])
    prange2 = np.array(t2['rawPrM'])
    B2 = np.array(t2['satClkBiasM'])
    PRN2 = np.array(t2['svid'])
    x1 = nr_p[i]
    x2 = nr_p[b]
    for j in range(10):
        x1,x2,_,_ = each_step2(x1,x2)
    nr1_aft[i] = x1
err = np.zeros(len(nr_p[:300]))
for i in range(len(nr_p[:300])):
    err[i] = np.linalg.norm(nr_p[i]-x_true[i])
err = err[np.isnan(err)!=True]
print(np.average(err))
err_1 = np.zeros(len(nr1_aft[:300]))
for i in range(len(nr1_aft[:300])):
    err_1[i] = np.linalg.norm(nr1_aft[i]-x_true[i])
err_1 = err_1[np.isnan(err_1)!=True]
print(np.average(err_1))

1047.201113611566
1126.4050637917785


In [40]:
nr1_aft = np.zeros_like(nr_p)
for i in range(len(times[:300])):
    t1 = time1[time1['millisSinceGpsEpoch'] == times[i]]
    t2 = time1[time1['millisSinceGpsEpoch'] == times[b]]
    X1 = np.array(t1['xSatPosM'])
    Y1 = np.array(t1['ySatPosM'])
    Z1 = np.array(t1['zSatPosM'])
    prange1 = np.array(t1['rawPrM'])
    B1 = np.array(t1['satClkBiasM'])
    PRN1 = np.array(t1['svid'])
    X2 = np.array(t2['xSatPosM'])
    Y2 = np.array(t2['ySatPosM'])
    Z2 = np.array(t2['zSatPosM'])
    prange2 = np.array(t2['rawPrM'])
    B2 = np.array(t2['satClkBiasM'])
    PRN2 = np.array(t2['svid'])
    x1 = nr_p[i]
    x2 = nr_p[b]
    for j in range(10):
        x1,x2,_,_ = each_step3(x1,x2)
    nr1_aft[i] = x1
err = np.zeros(len(nr_p[:300]))
for i in range(len(nr_p[:300])):
    err[i] = np.linalg.norm(nr_p[i]-x_true[i])
err = err[np.isnan(err)!=True]
print(np.average(err))
err_1 = np.zeros(len(nr1_aft[:300]))
for i in range(len(nr1_aft[:300])):
    err_1[i] = np.linalg.norm(nr1_aft[i]-x_true[i])
err_1 = err_1[np.isnan(err_1)!=True]
print(np.average(err_1))

1047.201113611566
1162.0563540300402


In [41]:
nr1_aft = np.zeros_like(nr_p)
for i in range(len(times[:300])):
    t1 = time1[time1['millisSinceGpsEpoch'] == times[i]]
    t2 = time1[time1['millisSinceGpsEpoch'] == times[b]]
    X1 = np.array(t1['xSatPosM'])
    Y1 = np.array(t1['ySatPosM'])
    Z1 = np.array(t1['zSatPosM'])
    prange1 = np.array(t1['rawPrM'])
    B1 = np.array(t1['satClkBiasM'])
    PRN1 = np.array(t1['svid'])
    X2 = np.array(t2['xSatPosM'])
    Y2 = np.array(t2['ySatPosM'])
    Z2 = np.array(t2['zSatPosM'])
    prange2 = np.array(t2['rawPrM'])
    B2 = np.array(t2['satClkBiasM'])
    PRN2 = np.array(t2['svid'])
    x1 = nr_p[i]
    x2 = nr_p[b]
    for j in range(10):
        x1,x2,_,_ = each_step4(x1,x2)
    nr1_aft[i] = x1
err = np.zeros(len(nr_p[:300]))
for i in range(len(nr_p[:300])):
    err[i] = np.linalg.norm(nr_p[i]-x_true[i])
err = err[np.isnan(err)!=True]
print(np.average(err))
err_1 = np.zeros(len(nr1_aft[:300]))
for i in range(len(nr1_aft[:300])):
    err_1[i] = np.linalg.norm(nr1_aft[i]-x_true[i])
err_1 = err_1[np.isnan(err_1)!=True]
print(np.average(err_1))

1047.201113611566
929.9246912665119


In [42]:
B = pd.DataFrame()

### Alternate approach

In [43]:
x_1 = np.zeros(3)
x0 = np.array([0,0,0])
nr1 = np.zeros((len(times),3))
nr2 = np.zeros((len(times),3))
for i in range(len(times)):
    a = time1[time1['millisSinceGpsEpoch'] == times[i]]
    b = a.copy()
    half = int(len(a)/2)
    if half>3:
        a = a[:half]
    X1 = np.array(a['xSatPosM'])
    Y1 = np.array(a['ySatPosM'])
    Z1 = np.array(a['zSatPosM'])
    prange1 = np.array(a['rawPrM'])
    B1 = np.array(a['satClkBiasM'])
    PRN1 = np.array(a['svid'])
    x_1, _,_,_,_ = solve_pos(x0, X1, Y1, Z1, B1, prange1)
    nr1[i] = x_1
    
    if half>3:
        a = b[half:]
    X1 = np.array(a['xSatPosM'])
    Y1 = np.array(a['ySatPosM'])
    Z1 = np.array(a['zSatPosM'])
    prange1 = np.array(a['rawPrM'])
    B1 = np.array(a['satClkBiasM'])
    PRN1 = np.array(a['svid'])
    x_1, _ ,_,_,_= solve_pos(x0, X1, Y1, Z1, B1, prange1)
    nr2[i] = x_1

In [44]:
nr1_aft = np.zeros_like(nr1)
for i in range(len(times[:300])):
    a = time1[time1['millisSinceGpsEpoch'] == times[i]]
    h = int(len(a)/2)
    t1 = a[:h]
    t2 = a[h:]
    X1 = np.array(t1['xSatPosM'])
    Y1 = np.array(t1['ySatPosM'])
    Z1 = np.array(t1['zSatPosM'])
    prange1 = np.array(t1['rawPrM'])
    B1 = np.array(t1['satClkBiasM'])
    PRN1 = np.array(t1['svid'])
    X2 = np.array(t2['xSatPosM'])
    Y2 = np.array(t2['ySatPosM'])
    Z2 = np.array(t2['zSatPosM'])
    prange2 = np.array(t2['rawPrM'])
    B2 = np.array(t2['satClkBiasM'])
    PRN2 = np.array(t2['svid'])
    x1 = nr1[i]
    x2 = nr2[i]
    for j in range(10):
        x1,x2,_,_ = each_step2(x1,x2)
    nr1_aft[i] = x1
err = np.zeros(len(nr1[:300]))
for i in range(len(nr1[:300])):
    err[i] = np.linalg.norm(nr1[i]-x_true[i])
err = err[np.isnan(err)!=True]
print(np.average(err))
err_1 = np.zeros(len(nr1_aft[:300]))
for i in range(len(nr1_aft[:300])):
    err_1[i] = np.linalg.norm(nr1_aft[i]-x_true[i])
err_1 = err_1[np.isnan(err_1)!=True]
print(np.average(err_1))

1239.6923020377005
1647.7097836769456
